In [45]:
'''
Do labeling only for mRNA puncta. 

if imaged proteins are not looking like puncta, then labeling of these proteins more likely does not make sense
'''



# creates a list of paths (path_list) with working files

import os

# Create a list of paths the name of which has two variables: mNRA_Name and SubSET
# def get_path(mNRA_Red, mRNA_Green, SubSET):

def get_path():


    input_dir = fr'\\?\G:\ISO\4h_second_round\Calm3'

    path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                
                # Read the file content and convert it to lowercase
                
                path_list.append(root)

    # Return the list of paths
    return path_list


In [46]:
# runs function get_path and saves path list as .txt file

path_list = get_path()

with open('path_list.txt', 'w') as file:
    for item in path_list:
        file.write(item + '\n')
        
len(path_list)

68

In [47]:
# function for parallelization that takes masked and raw images of mRNA from the path list and labels puncta


from concurrent.futures import ThreadPoolExecutor
from skimage import io
import os
import re
import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.filters import gaussian
from skimage.measure import label
import tifffile
from skimage.morphology import remove_small_objects
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure

# path_list = [fr'\\?\D:\Projects\LocProt\Hi-Plex\Leica\Cell1\aligned\ML_test\Analysis']

def normalize_image(file_path):
    image = tifffile.imread(file_path)

    # Convert to float for processing
    img_3d = image.astype(np.float32)  # Use float32 for memory efficiency

    # Normalize each image based on its own max value
    # max_val = img_3d.mean() * 5 # Cell Bodies
    max_val = img_3d.max() # mRNA and Protein 
    # max_val = img_3d.max() / 2 # Nuclei 
    
    if max_val > 0:  # Avoid division by zero
        img_3d /= max_val
        
    return img_3d
    
def process_image(path):
    
    mRNA_dir = fr'\\?\G:\ISO\4h_second_round\Calm3\Labeled'
    
    # Check if the directory exists, and create it if it does not
    if not os.path.exists(mRNA_dir):
        os.makedirs(mRNA_dir, exist_ok=True)
        
    
    with open(os.path.join(path, os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()

    # # Extract channel numbers and names using regular expressions
    # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+)', content)[0]))
    # channel_names = re.findall(r'ChannelNames:(\w+),(\w+)', content)[0]
    
    # # Extract channel numbers and names using regular expressions
    # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+)', content)[0]))
    # channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+)', content)[0]
    
    # Extract channel numbers and names using regular expressions
    channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+),(\d+)', content)[0])) #ONLY for 3rd channel
    channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+),(\w+)', content)[0] #ONLY for 3rd channel      

    # Combine channel numbers and names into a dictionary
    channel_info = dict(zip(channel_numbers, channel_names))

    mask0 = channel_names[0]
    raw0 = channel_numbers[0]
    mask1 = channel_names[1]
    raw1 = channel_numbers[1]
    mask2 = channel_names[2]
    raw2 = channel_numbers[2]
    mask3 = channel_names[3]
    raw3 = channel_numbers[3]
        
    # mRNA1_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask1}_filtered_image.tif')
    # mRNA2_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask2}_filtered_image.tif') 
    # mRNA3_Name_SubSET_mask_path = os.path.join(path, 'Segmented', f'Masked_{mask3}_filtered_image.tif')        

    Cytosol_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Cytosol_image.tif')
    Body_SubSET_mask_path = os.path.join(path, 'Segmented', 'Masked_Body_image.tif')    
    # raw1_image_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw1}.tif')
    # raw2_image_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw2}.tif')
    raw3_image_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw3}.tif')

    # raw_image1 = normalize_image(raw1_image_path)
    # raw_image2 = normalize_image(raw2_image_path)
    raw_image3 = normalize_image(raw3_image_path)

    cytosol = tifffile.imread(Cytosol_SubSET_mask_path)
    body = tifffile.imread(Body_SubSET_mask_path)
    
    # Apply a 3D Gaussian filter to the raw image
    # smoothed_image1 = gaussian(raw_image1, sigma=1.1)
    # smoothed_image2 = gaussian(raw_image2, sigma=0.5)
    smoothed_image3 = gaussian(raw_image3, sigma=1.1)

    # Threshold the smoothed image
    # threshold_value1 = 0.1 * np.max(smoothed_image1)
    # threshold_value2 = 0.1 * np.max(smoothed_image2)
    threshold_value3 = 0.1 * np.max(smoothed_image3)

    # thresholded_image1 = (smoothed_image1 > threshold_value1)
    # thresholded_image2 = (smoothed_image2 > threshold_value2)
    thresholded_image3 = (smoothed_image3 > threshold_value3)

    # Mask out any pixels that are not part of a signal
    # masked_image1 = thresholded_image1
    # masked_image2 = thresholded_image2
    masked_image3 = thresholded_image3

    # remove small objects (optional)
    min_size = 20 #in voxels
    # masked_image1 = remove_small_objects(masked_image1, min_size=min_size)
    # masked_image2 = remove_small_objects(masked_image2, min_size=min_size)
    masked_image3 = remove_small_objects(masked_image3, min_size=min_size)

    # filtered_masked_image1 = body * masked_image1
    # filtered_masked_image2 = body * masked_image2
    filtered_masked_image3 = body * masked_image3
    
    # masked_image1 = cytosol * masked_image1
    # masked_image2 = cytosol * masked_image2
    masked_image3 = cytosol * masked_image3
    
    # Compute the distance transform
    # distance1 = ndi.distance_transform_edt(masked_image1)
    # distance2 = ndi.distance_transform_edt(masked_image2)
    distance3 = ndi.distance_transform_edt(masked_image3)

    # Find the local maxima in the distance map
    # local_maxi1 = ndi.maximum_filter(distance1, size=5) == distance1
    # local_maxi2 = ndi.maximum_filter(distance2, size=5) == distance2
    local_maxi3 = ndi.maximum_filter(distance3, size=5) == distance3

    # Label each local maximum with a unique label
    # markers1 = label(local_maxi1)
    # markers2 = label(local_maxi2)
    markers3 = label(local_maxi3)

    # Apply the watershed algorithm to segment the signals
    # labeled_image1 = watershed(-distance1, markers1, mask=masked_image1)
    # labeled_image2 = watershed(-distance2, markers2, mask=masked_image2)
    labeled_image3 = watershed(-distance3, markers3, mask=masked_image3)

    directory_name = 'Segmented'
    segmented_dir_path = os.path.join(path, directory_name)

    # Add these lines
    if not os.path.exists(segmented_dir_path):
        os.makedirs(segmented_dir_path)

    #saves the segmented images in the same directory the processed images are
    # Labeled_mrna1_path = os.path.join(segmented_dir_path, f'Labeled_{mask1}.tif')
    # Masked_mrna1_path = os.path.join(segmented_dir_path, f'Masked_{mask1}_filtered.tif')
    # Labeled_mrna2_path = os.path.join(segmented_dir_path, f'Labeled_{mask2}.tif')
    # Masked_mrna2_path = os.path.join(segmented_dir_path, f'Masked_{mask2}_filtered.tif')
    Labeled_mrna3_path = os.path.join(segmented_dir_path, f'Labeled_{mask3}.tif')
    Masked_mrna3_path = os.path.join(segmented_dir_path, f'Masked_{mask3}_filtered.tif')
        
    # io.imsave(Labeled_mrna1_path, labeled_image1.astype('uint16'))
    # io.imsave(Masked_mrna1_path, filtered_masked_image1)
    # io.imsave(Labeled_mrna2_path, labeled_image2.astype('uint16'))
    # io.imsave(Masked_mrna2_path, filtered_masked_image2)
    io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
    io.imsave(Masked_mrna3_path, filtered_masked_image3)
    
    # print(f"Labeled image 1 saved in: {os.path.abspath(Labeled_mrna1_path)}")
    # print(f"Labeled image 2 saved in: {os.path.abspath(Labeled_mrna2_path)}")
    print(f"Labeled image 3 saved in: {os.path.abspath(Labeled_mrna3_path)}")

    # three channels. Change parameters accordingly
    # plot_titles = [ f'Raw {mask1} {raw_image1.shape[1]}', f'Binary {mask1} {raw_image1.shape[1]}', f'filtered {mask1} {raw_image1.shape[1]}', f'Raw {mask2}', f'Binary {mask2}', f'filtered {mask2}', f'Raw {mask3}', f'Binary {mask3}', f'filtered {mask3}']
    # images = [np.max(raw_image1, axis=0), np.max(filtered_masked_image1, axis=0), np.max(labeled_image1, axis=0), np.max(raw_image2, axis=0), np.max(filtered_masked_image2, axis=0), np.max(labeled_image2, axis=0), np.max(raw_image3, axis=0), np.max(filtered_masked_image3, axis=0), np.max(labeled_image3, axis=0)]

    # Two channels. Change parameters accordingly
    # plot_titles = [ f'Raw {mask1} {raw_image1.shape[1]}', f'Binary {mask1} {raw_image1.shape[1]}', f'filtered {mask2} {raw_image2.shape[1]}', f'Raw {mask2}', f'Binary {mask2}', f'filtered {mask2}']
    # images = [np.max(raw_image1, axis=0), np.max(filtered_masked_image1, axis=0), np.max(labeled_image1, axis=0), np.max(raw_image2, axis=0), np.max(filtered_masked_image2, axis=0), np.max(labeled_image2, axis=0)]

    # one channel. Change parameters accordingly
    plot_titles = [ f'Raw {mask3} {raw_image3.shape[1]}', f'filtered {mask3} {raw_image3.shape[1]}', f'Labeled {mask3} {raw_image3.shape[1]}']
    images = [np.max(raw_image3, axis=0), np.max(filtered_masked_image3, axis=0), np.max(labeled_image3, axis=0)]
        
    # Create a new figure and set the size
    fig = Figure(figsize=(10, 10))
    canvas = FigureCanvasAgg(fig)

    # Iterate through the images and add them as subplots
    for i, (image, title) in enumerate(zip(images, plot_titles)):
#             ax = fig.add_subplot(1, 1, i+1)
        ax = fig.add_subplot(3, 3, i+1)
        ax.imshow(image)
        ax.set_title(title)
        ax.axis('off')

    # Adjust the layout automatically
    fig.tight_layout()

    mRNA_path = os.path.join(mRNA_dir, os.path.basename(path) + '.png')

    # Save the plot as a PNG file with 300 DPI
    canvas.print_figure(mRNA_path, dpi=300)

    return

In [48]:
# function of parallelization of process_image() function

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def label_puncta_parallel(path_list):
    with ThreadPoolExecutor(max_workers=32) as executor: # specify the number of cpu cores you want to use
        futures = {executor.submit(process_image, path): path for path in path_list}

        for future in tqdm(as_completed(futures), total=len(path_list), desc="Processing Images"):
            pass


In [49]:
# this code creates paths to the folders where the segmentation was bad

import os

def get_paths_from_png_files_with_bad_segmentation():
    input_dir =  fr'\\?\G:\ISO\4h_second_round\Calm3\Again'

    # Check if the directory exists, and create it if it does not
    if not os.path.exists(input_dir):
        os.makedirs(input_dir, exist_ok=True)
        
    path_list = []

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if ".png" in file:
                # Remove the '.png' extension and the word 'Again' from the path
                modified_root = root.replace("Again", "")
                file_without_extension = os.path.splitext(file)[0]
                # Add the 'fr' prefix to the path when appending to path_list
                path_list.append(fr"{modified_root}{file_without_extension}")

    return path_list

# Get the path list with 'fr' prefix
path_list_with_bad_segmentation = get_paths_from_png_files_with_bad_segmentation()

# Print the path list to verify
for path in path_list_with_bad_segmentation:
    print(path)
print(len(path_list_with_bad_segmentation))

0


In [50]:
len(path_list)

68

In [51]:
len(path_list_with_bad_segmentation)

0

In [52]:
label_puncta_parallel(path_list)

Processing Images:   0%|                                                                        | 0/68 [00:00<?, ?it/s]C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Cacna1c-488_NCad-568_Calm3-647_c7_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))


Labeled image 3 saved in: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Cacna1c-488_NCad-568_Calm3-647_c7_deconv\Segmented\Labeled_Calm3.tif


C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Cacna1c-488_NCad-568_Calm3-647_c3_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Scn5a-488_NCad-568_Calm3-647_c9_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Cacna1c-488_NCad-568_Calm3-647_c6_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3

Labeled image 3 saved in: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Cacna1c-488_NCad-568_Calm3-647_c3_deconv\Segmented\Labeled_Calm3.tif


C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Camk2d-488_NCad-568_Calm3-647_c4_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Scn5a-488_NCad-568_Calm3-647_c3_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_ISO-4h_Camk2d-488_NCad-568_Calm3-647_c5_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
Processing Images:   3%|█▉                                                              | 2/68 [01:18<36:19, 33.0

...

Processing Images:  97%|█████████████████████████████████████████████████████████████▏ | 66/68 [02:49<00:02,  1.22s/it]C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_Saline-4h_Scn5a-488_NCad-568_Calm3-647_c8_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))


Labeled image 3 saved in: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_Saline-4h_Scn5a-488_NCad-568_Calm3-647_c8_deconv\Segmented\Labeled_Calm3.tif


Processing Images:  99%|██████████████████████████████████████████████████████████████ | 67/68 [02:56<00:03,  3.04s/it]C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:161: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_Saline-4h_Scn5a-488_NCad-568_Calm3-647_c7_deconv\Segmented\Labeled_Calm3.tif is a low contrast image
  io.imsave(Labeled_mrna3_path, labeled_image3.astype('uint16'))
C:\Users\Exx\AppData\Local\Temp\ipykernel_2228\3972601648.py:162: UserWarning: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_Saline-4h_Scn5a-488_NCad-568_Calm3-647_c7_deconv\Segmented\Masked_Calm3_filtered.tif is a low contrast image
  io.imsave(Masked_mrna3_path, filtered_masked_image3)


Labeled image 3 saved in: \\?\G:\ISO\4h_second_round\Calm3\2023-09-26_C57BL6_M_Saline-4h_Scn5a-488_NCad-568_Calm3-647_c7_deconv\Segmented\Labeled_Calm3.tif


Processing Images: 100%|███████████████████████████████████████████████████████████████| 68/68 [03:13<00:00,  2.85s/it]
